# Task 2: Method 2: Claim + Evidence Text Classification

In [ ]:
import pandas as pd
import numpy as np
import torch
import time
import json
import torch.nn as nn
import torch.optim as optim
from transformers import BertModel, BertTokenizer
from torch.utils.data import Dataset, DataLoader

## Model Training

In [ ]:
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

def transform_sentence_tran(sentence1, sentence2):
    
    maxlen = 256
    
    if not isinstance(sentence1, str):
        sentence1 = ""
    if not isinstance(sentence2, str):
        sentence2 = ""

    tokens1 = tokenizer.tokenize(sentence1)
    tokens2 = tokenizer.tokenize(sentence2)

    tokens = ['[CLS]'] + tokens1 + ['[SEP]'] + tokens2 + ['[SEP]']
    
    if len(tokens) < maxlen:
        tokens = tokens + ['[PAD]' for _ in range(maxlen - len(tokens))]
    else:
        tokens = tokens[:maxlen-1] + ['[SEP]']
    
    tokens_ids = tokenizer.convert_tokens_to_ids(tokens) 
    tokens_ids_tensor = torch.tensor(tokens_ids)
    
    attn_mask = (tokens_ids_tensor != 0).long()
    
    return tokens_ids_tensor, attn_mask

def transform_sentence_test(sentence1, sentence2):
    
    maxlen = 256
    
    if not isinstance(sentence1, str):
        sentence1 = ""
    if not isinstance(sentence2, str):
        sentence2 = ""

    tokens1 = tokenizer.tokenize(sentence1)
    tokens2 = tokenizer.tokenize(sentence2)

    tokens = ['[CLS]'] + tokens1 + ['[SEP]'] + tokens2 + ['[SEP]']
    
    if len(tokens) < maxlen:
        tokens = tokens + ['[PAD]' for _ in range(maxlen - len(tokens))]
    else:
        tokens = tokens[:maxlen-1] + ['[SEP]']
    
    tokens_ids = tokenizer.convert_tokens_to_ids(tokens) 
    tokens_ids_tensor = torch.tensor([tokens_ids])
    
    attn_mask = (tokens_ids_tensor != 0).long()
    
    return tokens_ids_tensor, attn_mask

def transform_label(label_string):
    if label_string == "SUPPORTS":
        label = torch.tensor([1,0,0,0])
    elif label_string == "REFUTES":
        label = torch.tensor([0,1,0,0])
    elif label_string == "DISPUTED":
        label = torch.tensor([0,0,1,0])
    else:
        label = torch.tensor([0,0,0,1])
    return label

def predict(output):
    index = np.argmax(output.tolist()[0])
    if index == 0:
        prediction = "SUPPORTS"
    elif index == 1:
        prediction = "REFUTES" 
    elif index == 2:
        prediction = "DISPUTED"
    else:
        prediction = "NOT_ENOUGH_INFO"
    return prediction

In [ ]:
class AFCDataset(Dataset):

    def __init__(self, filename):
        self.df = pd.read_csv(filename)
        self.tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

    def __len__(self):
        return len(self.df)

    def __getitem__(self, index):
        label = transform_label(self.df["label"][index])
        tokens_ids_tensor, attn_mask = transform_sentence_tran(self.df["claim"][index], self.df["evidence_text"][index])
        return tokens_ids_tensor, attn_mask, label

In [ ]:
tran_set = AFCDataset(filename = "data_processed/tran_df_t2.csv")
deva_set = AFCDataset(filename = "data_processed/deva_df_t2.csv")

tran_loader = DataLoader(tran_set, batch_size = 4)
deva_loader = DataLoader(deva_set, batch_size = 4)

In [ ]:
class ACF(nn.Module):

    def __init__(self):
        super(ACF, self).__init__()
        self.bert_layer = BertModel.from_pretrained('bert-base-uncased')
        self.cls_layer = nn.Linear(768, 4)

    def forward(self, seq, attn_masks):
        outputs = self.bert_layer(seq, attention_mask = attn_masks, return_dict=True)
        cont_reps = outputs.last_hidden_state
        cls_rep = cont_reps[:, 0]
        logits = self.cls_layer(cls_rep)
        return logits

In [ ]:
gpu = 0 
acf = ACF()
acf = acf.cuda(gpu) 

In [ ]:
criterion = nn.BCEWithLogitsLoss()
criterion = criterion.cuda()
opti = optim.Adam(acf.parameters(), lr = 2e-5)

In [ ]:
def train(net, criterion, opti, train_loader, dev_loader, max_eps, gpu):

    best_acc = 0
    st = time.time()
    for ep in range(max_eps):
        
        net.train()
        for it, (seq, attn_masks, labels) in enumerate(train_loader):
            opti.zero_grad()
            seq, attn_masks, labels = seq.cuda(gpu), attn_masks.cuda(gpu), labels.cuda(gpu)
            logits = net(seq, attn_masks)
            loss = criterion(logits.squeeze(-1), labels.float())
            loss.backward()
            opti.step()
            
            if it % 100 == 0:
                acc = get_accuracy_from_logits(logits, labels)
                print("Iteration {} of epoch {} complete. Loss: {}; Accuracy: {}; Time taken (s): {}".format(it, ep, loss, acc, (time.time()-st)))
                st = time.time()
        
        dev_acc, dev_loss = evaluate(net, criterion, dev_loader, gpu)
        print("Epoch {} complete! Development Accuracy: {}; Development Loss: {}".format(ep, dev_acc, dev_loss))
        if dev_acc > best_acc:
            print("Best development accuracy improved from {} to {}, saving model...".format(best_acc, dev_acc))
            best_acc = dev_acc
            torch.save(net.state_dict(), "t2_2.pth")

def get_accuracy_from_logits(logits, labels):
    probs = torch.sigmoid(logits.unsqueeze(-1))
    soft_probs = (probs > 0.5).long()
    acc = (soft_probs.squeeze() == labels).float().mean()
    return acc

def evaluate(net, criterion, dataloader, gpu):
    net.eval()
    mean_acc, mean_loss = 0, 0
    count = 0
    with torch.no_grad():
        for seq, attn_masks, labels in dataloader:
            seq, attn_masks, labels = seq.cuda(gpu), attn_masks.cuda(gpu), labels.cuda(gpu)
            logits = net(seq, attn_masks)
            mean_loss += criterion(logits.squeeze(-1), labels.float()).item()
            mean_acc += get_accuracy_from_logits(logits, labels)
            count += 1
    return mean_acc / count, mean_loss / count    
    

In [ ]:
num_epoch = 100

train(acf, criterion, opti, tran_loader, deva_loader, num_epoch, gpu)

## Model Evaluation

In [ ]:
model = ACF()
model.load_state_dict(torch.load("t2_2.pth"))

### Evaluation on Development Dataset

In [ ]:
deva_df = pd.read_csv("data_processed/deva_df_t2.csv")

In [ ]:
deva_pred_list = []
for i in range(len(deva_df)):
    tokens_ids_tensor, attn_mask = transform_sentence_test(deva_df["claim"][i], deva_df["evidence_text"][i])
    output = model(tokens_ids_tensor, attn_mask)
    prediction = predict(output)
    deva_pred_list.append(prediction)

In [ ]:
df_temp = pd.DataFrame({'Actual': list(deva_df["label"]), 'Predict': deva_pred_list, 'Values': list([1] * len(deva_pred_list))})
pivot_table = pd.pivot_table(df_temp, values='Values', index=['Actual'], columns=['Predict'], aggfunc='sum')
pivot_table

In [ ]:
evidence_index_list = []
for i in list(evdn_df["evidence_id"]):
    i = i.split(",")
    k = []
    for num in i:
        k.append(int(num))
    evidence_index_list.append(k)

with open("data_raw/dev-claims.json", 'r', encoding='utf8') as data:
    final_dict = json.load(data)

In [ ]:
index_list = list(deva_df["claim_index"])

for i in range(len(index_list)):
    index = index_list[i]
    evidence = []
    for evidence_index in evidence_index_list[i]:
        evidence.append("evidence-"+str(evidence_index))
    label = deva_pred_list[i]
    del final_dict["claim-"+str(index)]["claim_label"]
    final_dict["claim-"+str(index)]["claim_label"] = label
    del final_dict["claim-"+str(index)]["evidences"]
    final_dict["claim-"+str(index)]["evidences"] = evidence

In [ ]:
json_str = json.dumps(final_dict)
with open('deva-claims-predictions.json', 'w') as json_file:
    json_file.write(json_str)

### Evaluation on Testing Dataset

In [ ]:
test_df = pd.read_csv("data_processed/test_df_t2.csv")
evdn_df = pd.read_csv("evdn_pred/test_evdn_pred_tfidf_r2.csv")

In [ ]:
test_pred_list = []

for i in range(len(test_df)):
    tokens_ids_tensor, attn_mask = transform_sentence_test(test_df["claim"][i], evdn_df["evidence_text"][i])
    output = model(tokens_ids_tensor, attn_mask)
    prediction = predict(output)
    test_pred_list.append(prediction)


In [ ]:
evidence_index_list = []
for i in list(evdn_df["evidence_id"]):
    i = i.split(",")
    k = []
    for num in i:
        k.append(int(num))
    evidence_index_list.append(k)

with open("data_raw/test-claims-unlabelled.json", 'r', encoding='utf8') as data:
    final_dict = json.load(data)

index_list = list(test_df["claim_index"])

for i in range(len(index_list)):
    index = index_list[i]
    evidence = []
    for evidence_index in evidence_index_list[i]:
        evidence.append("evidence-"+str(evidence_index))
    label = test_pred_list[i]
    final_dict["claim-"+str(index)]["claim_label"] = label
    final_dict["claim-"+str(index)]["evidences"] = evidence

json_str = json.dumps(final_dict)
with open('test-claims-predictions.json', 'w') as json_file:
    json_file.write(json_str)